In [ ]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sts
import seaborn as sns

sns.set_style('white')

%matplotlib inline

dump_dir = ''
client = pymongo.MongoClient('localhost', 27017)
db = client.gamesdb

In [ ]:
db.collection_names()

## AI

In [ ]:
C = db.turing
cursor = C.find({'task':'AI'})

output_cols = ['user_name', 'opponent', 'color', 'bp', 'wp', 'response', 'rt', 'ts', 'status']

AI = pd.DataFrame(list(cursor))

In [ ]:
print(AI.user_name.unique())
AI.head()

In [ ]:
exclude = ['adfjk', 'bas', 'weiji', 'Ahmed']
AI = AI.loc[~AI.user_name.isin(exclude)].reset_index(drop=True)

In [ ]:
AI_out = AI.loc[AI.status != 'ready', output_cols]
AI_out['new_game'] = AI.bp.map(lambda x: int(np.array(list(x)).astype(int).sum()==1))
AI_out['last_move'] = AI_out['new_game'].shift(-1)
AI_out['status_code'] = AI_out.status.map(lambda x: 1 if x=='win' else 0)
AI_out['opponent'] = AI_out['opponent'].astype(int)
AI_out['opponent_bin'] = AI_out['opponent'] // 5
AI_out.to_csv(dump_dir + 'AI.csv')

In [ ]:
gl = np.loadtxt('/Users/maadmin/Downloads/game_length.txt') * 1.5
cursor = db.test_collection.find({'task': {'$in': ['turing']}})
T = pd.DataFrame(list(cursor))
exclude = ['bas', 'bvf', 'tes', 'test', 'gianni', 'dsfv', 'yunqi', 'adfjk', 'badsafgdsdg', 'don', 'Weiji', 'adfads', 'admin']
output_cols = ['user_name', 'task', 'clip_id', 'choice', 'feedback', 'timestamp', 'trial_start', 'experiment_start', 'start']
T = T.loc[~T.user_name.isin(exclude), output_cols].reset_index(drop=True)
T['feedback'] = T['feedback'].map(lambda x: 0 if x == 'false' else 1)
T['rt'] = (T['timestamp'].astype(int) - T['timestamp'].astype(int).shift(1))
T['game_length'] = gl[T['clip_id']]
T.to_csv(dump_dir + 'Turing.csv')
T.head()

## Turing figures

In [ ]:
clip_piv = T.pivot_table(index='clip_id', values='feedback', aggfunc=np.sum)

fig, axes = plt.subplots(1, 1, figsize=(10, 6))

axes.hist(clip_piv.values, normed=True, color='grey', edgecolor='white', bins=np.arange(0, 27, 1))
x = np.arange(0, 26, 1)
y = sts.binom.pmf(x, T['user_name'].unique().size, T['feedback'].mean())
axes.plot(x, y, color='black', label='binomial pmf, p = population mean')
plt.setp(axes, xlabel='Number of correct responses per clip', ylabel='Frequency of response')
axes.legend()
sns.despine()

In [ ]:
sub_piv = T.pivot_table(index='user_name', values='feedback', aggfunc=np.mean)
plt.hist(sub_piv.values, bins=np.arange(.4, .7, .025))
sns.despine()

In [ ]:
fd = T.loc[(T.user_name=='FD'), :]
gl[fd.clip_id.astype(int)]

In [ ]:
winfrac = AI_out.loc[AI_out.last_move==1, :].pivot_table(
    index='opponent_bin', values='status_code', aggfunc=np.mean
).sort_index()

winfrac.plot()
sns.despine()

# Turing

In [ ]:
fd['gl'] = gl[fd.clip_id.astype(int)]

plt.hist(fd.rt.values[1:]/1000 - fd.gl.values[1:], bins=np.arange(-10, 140, 5))
sns.despine()

In [ ]:
g = turing.loc[turing.user_name.isin(['Weiji']), :] # 'gianni'
g['correct'] = g['feedback'].map(lambda x: x=='true').astype(int)
g['correct'] = g['correct'].astype(int)
sts.bayes_mvs(g.correct, alpha=.95) #.loc[g.user_name=='BB', :]

In [ ]:
sts.bayes_mvs(g.loc[g.user_name=='FD', 'correct'].values.astype(int))

In [ ]:
g.loc[g.user_name=='FD', 'choice'].values.astype(int).size

In [ ]:
x = g.loc[g.user_name=='BB', 'choice'].values.astype(int)
y = g.loc[g.user_name=='gianni', 'choice'].values.astype(int)
print(sts.linregress(x, y))
plt.plot(
    x, 
    y, 
    linestyle='none', marker='o'
)

sns.despine()

In [ ]:
BB = g.loc[g.user_name=='BB', :]
print(sts.linregress(BB.choice.astype(int), BB.correct.astype(int)))

In [ ]:
x = np.abs(50-BB.loc[BB.correct==1, 'choice'].values.astype(int))
y = np.abs(50-BB.loc[BB.correct==0, 'choice'].values.astype(int))
plt.hist(np.abs(50-BB.loc[BB.correct==1, 'choice'].values.astype(int)), bins=np.arange(0, 51, 2), alpha=.8, normed=True)
plt.hist(np.abs(50-BB.loc[BB.correct==0, 'choice'].values.astype(int)), bins=np.arange(0, 51, 2), alpha=.8, normed=True)
sns.despine()

In [ ]:
sts.bayes_mvs(y)

In [ ]:
sts.bayes_mvs(x)

## One offs:

(correcting errors)

In [ ]:
# EXAMPLE: replace user_name field to be correct

F = {'user_name': }
U = {'$set': {'user_name': 'MJ'}}

C.update(F, U, multi=True)

In [ ]:
# EXAMPLE: remove test user from database

F = {'user_name': {'$in': ['gianni', 'bas', 'test']}}

C.remove(F, multi=True)